In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import contractions
from datasets import load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import torch

In [ ]:
# === Data Cleaning Functions ===
def clean_text(text):
    # Remove special characters and extra spaces, convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

def remove_urls(text):
    return re.sub(r'http\S+|www\S+|@\S+', '', text)

def expand_contractions(text):
    return contractions.fix(text)

In [ ]:
import pandas as pd

train_path = '/content/drive/MyDrive/train.csv'
test_path = '/content/drive/MyDrive/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(train_df.head())

train_df['Body_question'] = train_df['Body_question'].apply(lambda x: clean_text(remove_urls(expand_contractions(x))))
train_df['Body_answer'] = train_df['Body_answer'].apply(lambda x: clean_text(remove_urls(expand_contractions(x))))

test_df['Body_question'] = test_df['Body_question'].apply(lambda x: clean_text(remove_urls(expand_contractions(x))))
test_df['Body_answer'] = test_df['Body_answer'].apply(lambda x: clean_text(remove_urls(expand_contractions(x))))

   Unnamed: 0                                      Body_question  \
0        6604  I'm new to machine learning and I try to creat...   
1         106  I'm using Neural Networks to solve different M...   
2        2993  I have a training set composed of images havin...   
3       10766  I have encountered a strange situation where t...   
4        4315  I have trained a CNN model and I have applied ...   

                                         Body_answer  
0  You have two separate problems going on.\n\nUs...  
1  UPDATE: the landscape has changed quite a bit ...  
2  I have a suggestion for you. Maybe not complet...  
3  If I am getting you right, you are trying to p...  
4  Do you only have one single model? If you were...  


In [ ]:
# load data
dataset = load_dataset(
    'csv',
    data_files={
        'train': '/content/drive/MyDrive/train.csv',
        'test': '/content/drive/MyDrive/test.csv'
    }
)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Body_question', 'Body_answer'],
        num_rows: 9751
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Body_question', 'Body_answer'],
        num_rows: 2438
    })
})


In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codet5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [ ]:
# preprocess function
def preprocess_batch(examples):
    inputs = ["question: " + question for question in examples['Body_question']]
    targets = examples['Body_answer']
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(targets, truncation=True, padding="max_length", max_length=512)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_datasets = dataset.map(preprocess_batch, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/9751 [00:00<?, ? examples/s]

Map:   0%|          | 0/2438 [00:00<?, ? examples/s]

In [ ]:
# load model 
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="max_length",
    max_length=512
)

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
from transformers import EarlyStoppingCallback

output_dir = '/content/drive/My Drive/DSAN6600_final'

# define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=10,  
    weight_decay=0.01,  
    logging_dir=f'{output_dir}/logs',
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,
    logging_steps=500,
    eval_steps=500,
    label_smoothing_factor=0.1,  
    gradient_checkpointing=True  
)

# define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'].with_format('torch'),
    eval_dataset=tokenized_datasets['test'].with_format('torch'),
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Add Early Stopping
)

trainer.train()

# evaluate
results = trainer.evaluate()
print("Evaluation Results:", results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-1fbf419ee171>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,2.840100,2.777918
2,2.782000,2.732677
3,2.709700,2.708926
4,2.661000,2.692534
5,2.641100,2.682976
6,2.602700,2.677666
7,2.597400,2.674316
8,2.563400,2.672570
9,2.562200,2.671779
10,2.539100,2.671485


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Evaluation Results: {'eval_loss': 2.6714847087860107, 'eval_runtime': 33.1917, 'eval_samples_per_second': 73.452, 'eval_steps_per_second': 18.378, 'epoch': 10.0}


In [ ]:
# generation
def generate_output(prompt, model, tokenizer):
    # tokenize the input prompt
    input_ids = tokenizer("question: " + prompt, return_tensors="pt").input_ids.to(device)

    # generate the output with diverse decoding
    # topk + top sampling
    outputs = model.generate(
        input_ids=input_ids,
        max_length=256,
        do_sample=True,           
        top_k=30,                 
        top_p=0.9,                
        temperature=0.7,          
        repetition_penalty=1.5,   
        no_repeat_ngram_size=3   
    )

    # decode the generated output
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# prompt 
prompt = "Write a Python function to add two numbers."
generated_code = generate_output(prompt, model, tokenizer)
print("Generated Code:")
print(generated_code)

Generated Code:
In Python, you can do the following:
from scipy import optimize as np
import numpy as np # This will return a single integer with 0 and 1. If not there is any mathematical relationship between two numbers then it's possible that both are positive or negative (or something like this) to be represented by an int32 value. 

This would give us how much of your function works?  It may make sense to use python functions for adding values in some way but insteadof using strings directly from NumPy arrays which have more than 2 integers per iteration - e., g.: if len(a &gt;2): print('Adding': ',add_1), add_2([a, b]) else : pass

